### CLASSIFICAZIONE PURA

Lo scopo di questo lavoro è quello di classificare alcune recensioni Amazon attraverso metodi tradizionali di machine learning. 
In particolare, le rappresentazioni utilizzate sono BOW con pesi binari, frequenze e tf-idf ed un vocabolario costituito da 1-gram e 2-gram. 

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime > "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.4 gigabytes of available RAM

You are using a high-RAM runtime!


In [3]:
from time import time
import pandas as pd
import numpy as np
import random
import string
import os.path
import re

# Plot.
import seaborn as sns
import matplotlib.pyplot as plt

# NLP pipeline.
import nltk
nltk.download('stopwords')
stop_words = nltk.corpus.stopwords.words('english')
stop_words = stop_words + ['would'] + ['-PRON-'] # -PRON- viene aggiunto dal lemmatizer e va tolto.
import spacy # lemmatization
nlp = spacy.load('en', disable=['parser', 'ner'])

!pip install demoji
import demoji
demoji.download_codes()

# Rappresentazione.
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_selection import chi2
from sklearn.feature_selection import SelectPercentile

# Classificatori.
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, f1_score
from sklearn.model_selection import cross_validate

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
... OK (Got response in 1.41 seconds)
Writing emoji data to /root/.demoji/codes.json ...
... OK


In [4]:
# Codifica le sigle nei rispettivi classificatori. 
def decode(c):
  decoder = {'KNN': 'KNN', 
             'LSVC': 'Linear SVC', 
             'Logistic': 'Logistic', 
             'GNB': 'Gaussian Naive Bayes', 
             'Tree': 'Decision Tree Classifier', 
             'RF': 'Random Forest',
             'SGD':'Stochastic Gradient Descent Classifier',
             'MNB': 'Multinomial Naive Bayes', 
             'ADAB': 'Adaboost',
             'BNB': 'Bernullian Naive Bayes'}
  return decoder[c] 


# NLP pipeline.

def pipe(doc):
  doc = doc.lower()   # Lower case. 
  doc = re.sub(r'\d+', ' ', doc)  # Drop digits.
  doc = re.sub('['+string.punctuation+']', ' ', doc) # Drop punctuation. 
  doc = re.sub(r'\n+', ' ', doc) # Drop newline.
  if 'www.' in doc or 'http:' in doc or 'https:' in doc or '.com' in doc: # Drop URL.
    doc = re.sub(r"([^ ]+(?<=\.[a-z]{3}))", " ", doc)
  doc = demoji.replace(string = doc, repl = " ")  # Drop emoji.
  doc = re.sub(r'\s\s+', ' ', doc) # Drop extra spaces. 
  doc = nlp(doc)  
  doc = [token.lemma_ for token in doc]  # Tokenize and Lemmatization. 
  doc = [word for word in doc if len(word) > 2] # Drop short words.
  doc = [word for word in doc if word not in stop_words] # Drop stop words.
  return doc

In [5]:
# NLP pipeline: load, sample, clean dataset.

class preprocessing:
  # Initialize.
  def __init__(self, path, name_df):
    self.path = path
    self.name_df = name_df

  # Load data.
  def loader(self):
    print('IMPORT DATASET ' + self.name_df)
    if os.path.isfile(self.path) == False: 
      print('Set not exists.')
      raise SystemExit("Stop right there!")
    else:
      with open(self.path, encoding="utf8") as file:
       self.df = file.readlines()
    print(f"Size: {len(self.df)}")

 # Sampling.
  def sampling(self, size):
    print('SAMPLING ' + self.name_df)
    random.seed = 20201230
    self.df = random.sample(self.df, size)  
    print(f"Size: {len(self.df)}")

  # From list to data frame
  def data_frame(self):
    print('CREATE DATASET: REVIEWS - LABELS ' + self.name_df)
    X = []
    labels = []

    for rev in self.df:
      _, label, sent = re.split(r'__label__(\d)', rev)
      label = int(label[0]) - 1
      labels.append(label)
      X.append(sent)

    self.df = pd.DataFrame(list(zip(X, labels)), columns=['Review', 'Labels']) 

  # NLP pipeline.
  def pipeline(self):
    print('NLP pipeline ' + self.name_df)
    t1 = time()
    self.df["Clean_Review"] = self.df["Review"].apply(pipe)
    t2 = time()
    print(f"Time: {t2-t1}")

  # Empty review,
  def drop_empty_review(self):
    print('DROP EMPTY REVIEW ' + self.name_df)
    self.df.drop(self.df[self.df['Clean_Review'].map(len) == 0].index)

# Set of possible classification techniques.
class classification:
    # Initialize.
    def __init__(self, X_train, y_train, cv):
        self.seed = 20201230
        self.X_train = X_train
        self.y_train = y_train
        self.cv = cv

    # Select the model.
    def select_model(self, class_type):
      possible_class = ['KNN', 'LSVC', 'Logistic', 'Tree', 'RF', 'SGD', 'MNB', 'ADAB', 'BNB']

      if class_type in possible_class:
        if class_type == possible_class[0]:
          clf = KNeighborsClassifier(n_neighbors = 5)
        if class_type == possible_class[1]:
          clf = LinearSVC(random_state=0, max_iter=1000)
        if class_type == possible_class[2]:
          clf = LogisticRegression(random_state=0, max_iter=1000)
        if class_type == possible_class[3]:
          clf = DecisionTreeClassifier()
        if class_type == possible_class[4]:
          clf = RandomForestClassifier()
        if class_type == possible_class[5]:
          clf = SGDClassifier(max_iter=1000, tol=1e-3)
        if class_type == possible_class[6]:
          clf = MultinomialNB()
        if class_type == possible_class[7]:
          clf = AdaBoostClassifier(n_estimators=100, random_state=0)
        if class_type == possible_class[8]:
          clf = BernoulliNB()

        self.clf = clf
        # Cross validation.
        self.score = cross_validate(self.clf, self.X_train, self.y_train, cv=self.cv, scoring = ['accuracy', 'f1'])
        self.N = int(self.X_train.shape[0]/cv)
        # Performances.
        perf = []
        for key in self.score:          
          perf.append([key, self.score[key].mean(), self.score[key].std() * 2/np.sqrt(self.N)])

        perf = np.array(perf)
        perf =  pd.DataFrame(perf, columns = ['Measure', 'Mean', 'StandError'])
        perf['Measure'] = ['TrainTime', 'TestTime', 'F1', 'Accuracy']
        print(perf)
        self.perf = perf
      else:
        raise SystemExit("Select one of the provided classifiers!")


### ***IMPORT AND PREPROCESSING***

In [6]:
 %%time

        #-- IMPORT SET.

train_path = 'drive/MyDrive/Text Mining/train.ft.txt'

train = preprocessing(train_path, 'TRAIN')

train.loader()

      #-- SAMPLING.

train_sample_size = 250000
train.sampling(train_sample_size)

      #-- CREATE A DATASET WITH REVIEW - LABEL.

train.data_frame()

      #-- PREPROCESSING.

train.pipeline()

      #-- DROP EMPTY REVIEWS.

train.drop_empty_review()

IMPORT DATASET TRAIN
Size: 3600000
SAMPLING TRAIN
Size: 250000
CREATE DATASET: REVIEWS - LABELS TRAIN
NLP pipeline TRAIN
Time: 1982.5033721923828
DROP EMPTY REVIEW TRAIN
CPU times: user 33min 3s, sys: 6.36 s, total: 33min 9s
Wall time: 33min 30s


### ***FEATURE EXTRACTION - SELECTION - WEIGHTED***


In [7]:
# Number of min and max gram.
gram_min = 1
gram_max = 2

#### ***BINARY MATRIX***

In [8]:
%%time

    #-- FEATURE EXTRACTION: BINARY BOW.

def dummy(doc):
  return doc

vectorizer = CountVectorizer(
        tokenizer=dummy,
        preprocessor=dummy,
        min_df = 0.0001, # 0.0001*100000 = 10, ragionevole
        # max_df = 0.8, # stop words removal: it's useless if I had removed stop words!
        ngram_range = (gram_min, gram_max),
        binary = True
    )  

train_BOW = vectorizer.fit_transform(train.df['Clean_Review'])

print(train_BOW.shape)

      #-- FEATURE SELECTION

select_percentile = SelectPercentile(chi2, percentile = 10)
train_BOW = select_percentile.fit_transform(train_BOW, train.df['Labels'])

print(train_BOW.shape)

(250000, 49651)
(250000, 4965)
CPU times: user 41.8 s, sys: 1.55 s, total: 43.4 s
Wall time: 43.3 s


##### ***CLASSIFICATION***



In [10]:
%%time

cv = 5
learn = classification(train_BOW, train.df['Labels'], cv)

my_class = ['KNN', 'LSVC', 'Logistic', 'SGD', 'MNB', 'ADAB', 'BNB']

for c in my_class:
  print(f"                                                            ***** {decode(c)} ****")
  learn.select_model(c)

# del train_BOW, test_BOW, learn

                                                            ***** KNN ****
     Measure                  Mean              StandError
0  TrainTime  0.056425857543945315   2.753943982440745e-05
1   TestTime      405.426735830307    0.028792811364903085
2         F1              0.693416  1.6904967317330087e-05
3   Accuracy    0.7055746159637808  1.4800327564727049e-05
                                                            ***** Linear SVC ****


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


     Measure                 Mean              StandError
0  TrainTime    47.31369495391846   0.0022346984335219968
1   TestTime  0.03417601585388184   2.692033779321446e-06
2         F1             0.892408  1.5533849490708952e-05
3   Accuracy   0.8933637621278171  1.5140168834118887e-05
                                                            ***** Logistic ****
     Measure                  Mean              StandError
0  TrainTime    12.047234106063843    0.002817145313543353
1   TestTime  0.033413267135620116   7.134668690415614e-06
2         F1              0.892984  1.2695775675397014e-05
3   Accuracy    0.8937880497236119  1.2310704634865834e-05
                                                            ***** Stochastic Gradient Descent Classifier ****
     Measure                 Mean              StandError
0  TrainTime   1.1151763439178466    0.000486505579879718
1   TestTime  0.03344082832336426   4.446155058648363e-06
2         F1             0.889756  1.52258306834142

#### ***FREQUENCY MATRIX***

In [11]:
%%time

    #-- FEATURE EXTRACTION: FREQUENCY BOW.
def dummy(doc):
  return doc

vectorizer = CountVectorizer(
        tokenizer=dummy,
        preprocessor=dummy,
        min_df = 0.0001, # 0.001*100000 = 100, ragionevole
        # max_df = 0.7,
        ngram_range = (gram_min, gram_max),
        binary = False,
        # vocabulary = my_dic
    )  

# Restituisce un amatrice di TIPO sparso quindi non bisogna convertirla. 
train_freq = vectorizer.fit_transform(train.df['Clean_Review'])

print(train_freq.shape)

      #-- FEATURE SELECTION

select_percentile = SelectPercentile(chi2, percentile=10)
train_freq = select_percentile.fit_transform(train_freq, train.df['Labels'])

print(train_freq.shape)

(250000, 49651)
(250000, 4965)
CPU times: user 43.3 s, sys: 931 ms, total: 44.3 s
Wall time: 44.2 s


##### ***CLASSIFICATION***

In [12]:
%%time

learn = classification(train_freq, train.df['Labels'], cv)

my_class = ['KNN', 'LSVC', 'Logistic', 'SGD', 'MNB', 'ADAB', 'BNB']

for c in my_class:
  print(f"                                                            ***** {decode(c)} ****")
  learn.select_model(c)

                                                            ***** KNN ****
     Measure                  Mean              StandError
0  TrainTime  0.051820564270019534  3.0159343966696418e-05
1   TestTime    424.34550976753235    0.031770297823593995
2         F1              0.695364   1.644322109563688e-05
3   Accuracy     0.712854534764533   1.136842365313508e-05
                                                            ***** Linear SVC ****


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


     Measure                  Mean              StandError
0  TrainTime     49.95259823799133     0.00764890861470465
1   TestTime  0.032660055160522464  5.7941280929614035e-06
2         F1               0.89284  1.0676328957090177e-05
3   Accuracy    0.8938462416191297  1.0532332214564921e-05
                                                            ***** Logistic ****
     Measure                 Mean              StandError
0  TrainTime    16.73496298789978    0.009529439249489648
1   TestTime  0.03167815208435058  3.2032122764567216e-06
2         F1               0.8939  1.0141400297789258e-05
3   Accuracy   0.8947499851691157   9.939340412747645e-06
                                                            ***** Stochastic Gradient Descent Classifier ****
     Measure                 Mean              StandError
0  TrainTime   1.2920207500457763   0.0005322219477875733
1   TestTime  0.03147468566894531  3.3296540341082516e-06
2         F1             0.890184  5.51756468018288

#### ***TF-IDF***

In [13]:
%%time

    #-- TF-IDF

def dummy(doc):
  return doc

vectorizer = TfidfVectorizer(
        tokenizer=dummy,
        preprocessor=dummy,
        min_df = 0.0001, # 0.001*100000 = 100, ragionevole
        # max_df = 0.7,
        ngram_range = (gram_min, gram_max),
        binary = False,
        # vocabulary = my_dic
    )  

train_tf_idf = vectorizer.fit_transform(train.df['Clean_Review'])

print(train_tf_idf.shape)

      #-- FEATURE SELECTION

select_percentile = SelectPercentile(chi2, percentile=10)
train_tf_idf = select_percentile.fit_transform(train_tf_idf, train.df['Labels'])

print(train_tf_idf.shape)

(250000, 49651)
(250000, 4965)
CPU times: user 43.6 s, sys: 844 ms, total: 44.5 s
Wall time: 44.4 s


##### CLASSIFICATION

In [14]:
%%time

learn = classification(train_tf_idf, train.df['Labels'], cv)

my_class = ['KNN', 'LSVC', 'Logistic', 'SGD', 'MNB', 'ADAB', 'BNB']

for c in my_class:
  print(f"                                                            ***** {decode(c)} ****")
  learn.select_model(c)


                                                            ***** KNN ****
     Measure                Mean             StandError
0  TrainTime  0.0501004695892334  5.531554973090335e-06
1   TestTime  360.98127636909487     0.1527442466782814
2         F1  0.6731040000000001  3.004069772824845e-05
3   Accuracy   0.669575876158963  5.555106304403867e-05
                                                            ***** Linear SVC ****
     Measure                 Mean              StandError
0  TrainTime   2.6826063632965087  0.00037259812446662383
1   TestTime  0.03076624870300293   3.893558038888187e-06
2         F1   0.8942439999999999  1.0251493549722402e-05
3   Accuracy   0.8948552432109123  1.0010236064449383e-05
                                                            ***** Logistic ****
     Measure                  Mean             StandError
0  TrainTime     6.419427919387817   0.002759457574639326
1   TestTime  0.029975175857543945  2.117509640714302e-06
2         F1    0.8